#### Mount Azure Data Lake using Service Principle
1. Get client_ID, tenant_id and client_secret from key vault
1. Set Spark config with app/client ID, directory/tenant ID and Secret
1. Call file system utility mount

In [0]:
def mount_adls(storage_account, container_name):
    
    # Get secrets from Azure vault
    client_secret = dbutils.secrets.get("formula1_scope", 'service-principle-key')
    client_id = dbutils.secrets.get("formula1_scope", 'client-id')
    tenant_id = dbutils.secrets.get("formula1_scope", 'tenant-id')

    # Set Spark config
    configs = {"fs.azure.account.auth.type": "OAuth",
               "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
               "fs.azure.account.oauth2.client.id": client_id,
               "fs.azure.account.oauth2.client.secret": client_secret,
               "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}
    
    # Check if container is already mounted
    if any(mount.mountPoint == f"/mnt/{storage_account}/{container_name}" for mount in dbutils.fs.mounts()):
        dbutils.fs.unmount(f"/mnt/{storage_account}/{container_name}")

    # Mount the container from specified storage account
    dbutils.fs.mount(
        source = f"abfss://{container_name}@{storage_account}.dfs.core.windows.net/",
        mount_point = f"/mnt/{storage_account}/{container_name}",
        extra_configs = configs)
    

#### Mount containers

In [0]:
# List of containers
containers = ["raw", "processed", "presentation"]

# Loop though them and mount
for container in containers:
    mount_adls("udemycourseproject", container)
    
display(dbutils.fs.mounts())

/mnt/udemycourseproject/raw has been unmounted.
/mnt/udemycourseproject/processed has been unmounted.
/mnt/udemycourseproject/presentation has been unmounted.


mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/mnt/udemycourseproject/raw,abfss://raw@udemycourseproject.dfs.core.windows.net/,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/udemycourseproject/demo,abfss://demo@udemycourseproject.dfs.core.windows.net/,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/udemycourseproject/processed,abfss://processed@udemycourseproject.dfs.core.windows.net/,
/,DatabricksRoot,
/mnt/udemycourseproject/presentation,abfss://presentation@udemycourseproject.dfs.core.windows.net/,


In [0]:
dbutils.secrets.list('formula1_scope')

Out[3]: [SecretMetadata(key='client-id'),
 SecretMetadata(key='notebook1-access-key'),
 SecretMetadata(key='sas-secret'),
 SecretMetadata(key='service-principle-key'),
 SecretMetadata(key='tenant-id')]

In [0]:
display(dbutils.fs.ls("/mnt/udemycourseproject/processed"))

path,name,size,modificationTime
dbfs:/mnt/udemycourseproject/processed/circuits/,circuits/,0,1687468601000
dbfs:/mnt/udemycourseproject/processed/races/,races/,0,1687475505000
